In [4]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import sys
import pandas as pd




In [17]:
# fileNames = ['predictions_SVR_RBF_TotalCost.csv',\
#             'predictions_eRNN_TotalCost.csv',\
#             'predictions_MLP_TotalCost.csv',
#             'predictions_LSTM_TotalCost.csv',]
fileNames = ['predictions_MLP_server1_tactic1.csv',\
            'predictions_MLP_server2_tactic1.csv',\
            'predictions_MLP_server3_tactic1.csv',]

In [19]:
inputFolder = '/Users/aizazulhaq/Desktop/RESEARCH/TVA-Results/PingPredictionResults/'
# dataPred = pd.read_csv('../Prediction_Files/arimaNormalizedPrediction.csv')
outputDirectory = '/Users/aizazulhaq/Desktop/RESEARCH/TVA-Results/CELIA-Results-Ping/'
outputFileName = outputDirectory + "rewardAwareUtility-SVR_Linear.csv"

# reading predictions and transforming data

In [7]:
def rowToTime(row):
    timeAtMoment = row.Hours*3600*1000
    timeAtMoment += row.Minutes*60*1000
    timeAtMoment += row.Seconds*1000
    return timeAtMoment

In [8]:
def shiftTransformCol50(data):
    return (data + abs(data.min()))*50

In [9]:
def utilFunction(latency, cost, reward = 100, reliability = 1):
    epsilon = 0.00000000000000000000000000000000000000000000001

    if reliability == 0:
        return 0
    else:
        return ((reward)/(latency+cost+epsilon)) * reliability 
    


realUtilityMean = 4.138921e+06	
realUtilStd = 6.175693e+06	

In [10]:
def celia(data,updateThreshold = 1000):
    lastUpdate = 0;
    utilitPredicted = []
    utilityReal = []
    award = []
    lostUtility = []
    shouldHaveUpdated = []
    performedUpdate = []

    temp = data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp']
    numOfUpdates = 0
    negativeAward = []
    for i in range(1,len(data)):
#     timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[i-1]['Timestamp']).total_seconds()
        timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[lastUpdate]['Timestamp'])

        predictedUtil = utilFunction(data.iloc[i].predLatency, data.iloc[i].predCost,(float)(timeDeltaReward), data.iloc[i].predReliability )

        realUtility = utilFunction(data.iloc[i].Latency, data.iloc[i].Cost,(float)(timeDeltaReward), data.iloc[i].Reliability )
        
        utilitPredicted.append(predictedUtil)
        
        utilityReal.append(realUtility)
        
        
        if predictedUtil >= updateThreshold:
    #         award.append(award[i-1] + realUtility-1);
#             award.append(realUtility-updateThreshold);

            lastUpdate = i;
            lostUtility.append(0)
            
            performedUpdate.append(1) #this is the should/not have updated checks 
            
            if realUtility >= updateThreshold:
                award.append(realUtility-updateThreshold);
                negativeAward.append(0)
                shouldHaveUpdated.append(1)
            else:
                award.append(0)
                shouldHaveUpdated.append(0)
                negativeAward.append(abs(realUtility-updateThreshold));
                
            numOfUpdates += 1
        else:
    #         award.append(award[i-1]);
            award.append(0);
            
            performedUpdate.append(0)
            
            if realUtility >= updateThreshold:
                lostUtility.append(realUtility)
                negativeAward.append(realUtility)

                shouldHaveUpdated.append(1)

            else:
                lostUtility.append(0) 
                
                negativeAward.append(0)

                shouldHaveUpdated.append(0)

    
    results = pd.DataFrame(np.stack([utilitPredicted,utilityReal, award, negativeAward, lostUtility,shouldHaveUpdated,performedUpdate],axis=1),columns=['predictedUtility','realUtility','reward','negativeReward','lostUtility','shouldHaveUpdated','performedUpdate'])
    return results, numOfUpdates
#     results.to_csv("./results/dummyRandom.csv")


In [11]:

def celiaSim(dataPred, updateThreshold = 1000, returnDataOnly = False):
    if returnDataOnly == True:
        print("returning data only")
#         return newData

    ### this is converting hours, minutes, seconds to a millisecond time series
    dataPred.Hours = ((dataPred.Hours)*23)+0
    dataPred.Minutes = ((dataPred.Minutes)*59)+0
    dataPred.Seconds = ((dataPred.Seconds)*59.999805)+0.1025061111111
    
    cumulativeTime = [rowToTime(dataPred.iloc[0])]
    resetAddTime = 23*3600*1000
    resetAddTime += 59*60*1000
    resetAddTime += 60*1000
        
    for i in range(1,len(dataPred)):
        timeAtMoment = rowToTime(dataPred.iloc[i])
        lastTimeMoment = rowToTime(dataPred.iloc[i-1])
        if timeAtMoment > lastTimeMoment:
            cumuTime = cumulativeTime[i-1] + (timeAtMoment - lastTimeMoment)
            cumulativeTime.append(cumuTime)
        else:
            cumulativeTime.append(cumulativeTime[i-1]+(timeAtMoment+resetAddTime)-lastTimeMoment)
    #         print(cumulativeTime[i])
    
    cumulativeTime = cumulativeTime- cumulativeTime[0]
    
    ### shifting and transforming data to a range of 0-50
    newData = np.stack((cumulativeTime, shiftTransformCol50(dataPred.Latency), shiftTransformCol50(dataPred.TotalCost), dataPred.Reliability, shiftTransformCol50(dataPred.predicted_Latency), shiftTransformCol50(dataPred.predicted_TotalCost), round(dataPred.predicted_Reliability)),axis = 1)
    newData = pd.DataFrame(newData, columns=['Timestamp', 'Latency', 'Cost', 'Reliability', 'predLatency', 'predCost', 'predReliability' ])
    
    if returnDataOnly == True:
        print("returning data only")
        return newData
    else:
        return celia(newData, updateThreshold)
#     return newData

In [12]:
# result = celiaSim(dataPred)


In [13]:
def predictionStatAnalysis(data):
    cost = abs(data.Cost - data.predicted_Cost).describe()
    latency = abs(data.Latency - data.predicted_Latency).describe()
    reliability = abs(data.Reliability - data.predicted_Reliability).describe()
    for i in range(1,len(cost)):
        print (cost[i])
    print('\n')
    for i in range(1,len(cost)):
        print (latency[i])

In [14]:
# result.to_csv(outputDirectory+'result_'+i)


In [15]:
celiaStats = []


In [20]:
predictionStats = []
for i in fileNames:
    dataPred = pd.read_csv(inputFolder+i)
    result, numOfUpdates = celiaSim(dataPred, 154)
    result.to_csv(outputDirectory+'celiaResult_'+i)
    utilDif = np.sum(abs(result.predictedUtility-result.realUtility))
    reward = sum(result.reward)
    negativeReward = sum(result.negativeReward)
    lostUtil = sum(result.lostUtility)
    
    decisionShouldHaveDid = len(result[result.shouldHaveUpdated == 1][result.performedUpdate == 1])
    decisionShouldHaveDidNot = len(result[result.shouldHaveUpdated == 1][result.performedUpdate == 0])
    decisionNotHaveDid = len(result[result.shouldHaveUpdated == 0][result.performedUpdate == 1])
    decisionNotHaveDidNot = len(result[result.shouldHaveUpdated == 0][result.performedUpdate == 0])
    
    wrongDecisions = decisionShouldHaveDidNot + decisionNotHaveDid
    correctDecisions = decisionShouldHaveDid + decisionNotHaveDidNot
    
    celiaStats.append([i, utilDif, reward, negativeReward, numOfUpdates, lostUtil, wrongDecisions, correctDecisions, decisionShouldHaveDid, decisionShouldHaveDidNot, decisionNotHaveDid, decisionNotHaveDidNot])
    

AttributeError: 'DataFrame' object has no attribute 'Hours'

In [68]:
celiaStats = pd.DataFrame(celiaStats, columns=['predictionType','utilDif','reward','negativeReward', 'numOfUpdates','lostUtil', 'wrongDecisions', 'correctDecisions', 'decisionShouldHaveDid', 'decisionShouldHaveDidNot', 'decisionNotHaveDid', 'decisionNotHaveDidNot'])
celiaStats.to_csv(outputDirectory+'totalCostStats.csv')


In [69]:
celiaStats

,predictionType,utilDif,reward,negativeReward,numOfUpdates,lostUtil,wrongDecisions,correctDecisions,decisionShouldHaveDid,decisionShouldHaveDidNot,decisionNotHaveDid,decisionNotHaveDidNot
0,predictions_SVR_RBF.csv,1.367645e+11,1.436975e+11,5.831630e+06,33072,5.829002e+06,18809,33287,33054,18791,18,233
1,predictions_MLP.csv,3.568253e+10,1.436530e+11,5.585436e+05,49316,5.558813e+05,2443,49653,49295,2422,21,358
2,predictions_LSTM.csv,5.015203e+10,1.436529e+11,1.075698e+06,47136,1.072504e+06,4645,47451,47104,4613,32,347
3,predictions_ARIMA.csv,1.371820e+11,1.436522e+11,7.379129e+06,33527,7.376501e+06,18226,33870,33509,18208,18,361


In [109]:
## looking for a good threshold based on the ideal class split
dataPred = celiaSim(pd.read_csv(inputFolder+fileNames[1]), 1, returnDataOnly=True)
bestCombo = {}

returning data only
returning data only


In [118]:
for i in  range(230, 240, 1):
    result,numberOfUpdates = celia(dataPred, i)
    counts = np.unique(result.shouldHaveUpdated,return_counts=True)[1]
    ratio = counts[0]/(counts[1]+counts[0])
    
    bestCombo[i] = ratio
    print (ratio)

0.25324401105651106
0.2749539312039312
0.30368934275184273
0.32017813267813267
0.33311578624078625
0.3393926597051597
0.34440264127764125
0.35000767813267813
0.3536739864864865
0.35657248157248156


In [117]:
bestCombo

{150: 0.006929514742014742,
 160: 0.007562960687960688,
 170: 0.008253992628992629,
 180: 0.01165156633906634,
 190: 0.01608568796068796,
 200: 0.02583691646191646,
 210: 0.04317030098280098,
 220: 0.07474662162162163,
 230: 0.25324401105651106,
 240: 0.36116016584766586,
 250: 0.3772266584766585,
 260: 0.3845208845208845,
 270: 0.3892237407862408,
 280: 0.3940609643734644,
 290: 0.39715141277641275,
 300: 0.39864864864864863,
 310: 0.3993972665847666,
 320: 0.40010749385749383,
 330: 0.4003762285012285,
 340: 0.4006065724815725,
 350: 0.4006833538083538,
 360: 0.4008177211302211,
 370: 0.40072174447174447,
 380: 0.4005681818181818,
 390: 0.4006257678132678,
 400: 0.40070254914004916,
 410: 0.40072174447174447,
 420: 0.4007409398034398,
 430: 0.4006065724815725,
 440: 0.4006833538083538,
 450: 0.4007985257985258,
 460: 0.4007985257985258,
 470: 0.4007409398034398,
 480: 0.40072174447174447,
 490: 0.40076013513513514,
 500: 0.40072174447174447,
 510: 0.40072174447174447,
 520: 0.4006257

In [78]:
counts = np.unique(result.shouldHaveUpdated,return_counts=True)[1]

In [83]:
counts[0]/(counts[1]+counts[0])

0.007275030712530712

In [46]:
celiaStats = []
predictionStats = []
bestCombo = (1000, 5437)
# for i in fileNames:
dataPred = pd.read_csv(inputFolder+'predictions_eRNN_TotalCost.csv')
newData = celiaSim(dataPred)

print(newData.describe())
for j in range(100,10000):
    result = celia(newData,j)
#             result.to_csv(outputDirectory+'celiaResult_'+i)

    utilDif = np.sum(abs(result.predictedUtility-result.realUtility))
    award = sum(result.award)
    lostUtil = sum(result.lostUtility)

    decisionShouldHaveDid = len(result[result.shouldHaveUpdated == 1][result.performedUpdate == 1])
    decisionShouldHaveDidNot = len(result[result.shouldHaveUpdated == 1][result.performedUpdate == 0])
    decisionNotHaveDid = len(result[result.shouldHaveUpdated == 0][result.performedUpdate == 1])
    decisionNotHaveDidNot = len(result[result.shouldHaveUpdated == 0][result.performedUpdate == 0])

    print (decisionShouldHaveDidNot+decisionNotHaveDid)

    if decisionShouldHaveDidNot+decisionNotHaveDid <bestCombo[1]:
        bestCombo = (j,decisionShouldHaveDidNot+decisionNotHaveDid )
                
#     celiaStats.append([i, utilDif, award, lostUtil, decisionShouldHaveDid, decisionShouldHaveDidNot, decisionNotHaveDid, decisionNotHaveDidNot])
    


          Timestamp       Latency          Cost  Reliability   predLatency  \
count  1.310600e+04  13106.000000  13106.000000      13106.0  13106.000000   
mean   1.764051e+10      1.419579     12.884231          1.0      2.107865   
std    1.045092e+10      1.473449     14.233377          0.0      0.182612   
min    0.000000e+00      0.151978      0.265145          1.0      1.765920   
25%    8.567929e+09      0.205932      1.779819          1.0      2.002526   
50%    1.799972e+10      0.923482      3.618440          1.0      2.076285   
75%    2.674000e+10      2.565513     20.175260          1.0      2.246980   
max    3.600034e+10     49.315339     49.806473          1.0      3.170165   

           predCost  predReliability  
count  13106.000000          13106.0  
mean      13.223633              1.0  
std        1.063214              0.0  
min       10.997400              1.0  
25%       12.511762              1.0  
50%       13.124300              1.0  
75%       13.860725     

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


4427
4425
4426
4426
4425
4423
4421
4420
4420
4420
4420
4421
4421
4421
4420
4421
4423
4423
4423
4421
4418
4420
4419
4418
4419
4415
4409
4407
4407
4417
4403
4398
4397
4389
4381
4367
4351
4322
4298
4275
4252
4225
4137
3963
3744
3601
3077
2892
2377
2127
1556
1095
883
847
843
867
902
972
1062
1177
1370
1698
2016
2282
2711
3370
3998
4423
4641
4809
4949
4999
5053
5086
5099
5111
5124
5141
5156
5167
5180
5201
5217
5228
5246
5261
5271
5291
5291
5304
5308
5313
5311
5302
5298
5281
5274
5253
5234
5205
5159
5112
5085
5034
4991
4945
4871
4792
4719
4642
4570
4500
4418
4354
4264
4208
4148
4084
4025
3962
3909
3840
3793
3759
3716
3686
3652
3621
3587
3550
3507
3475
3441
3411
3396
3376
3360
3355
3344
3352
3352
3349
3354
3366
3382
3407
3448
3488
3569
3647
3737
3848
3986
4136
4291
4444
4564
4673
4748
4804
4848
4883
4911
4922
4941
4959
4988
4997
5010
5023
5041
5057
5071
5085
5095
5102
5105
5117
5120
5129
5136
5141
5142
5154
5157
5162
5162
5167
5168
5169
5174
5180
5179
5186
5189
5195
5201
5204
5212
5214
5219
5

6367
6368
6368
6367
6364
6363
6364
6361
6362
6365
6363
6361
6361
6361
6360
6362
6358
6357
6355
6358
6360
6357
6357
6355
6356
6357
6364
6365
6357
6359
6361
6358
6353
6351
6344
6340
6340
6342
6341
6342
6342
6341
6335
6334
6342
6340
6339
6334
6338
6337
6341
6342
6343
6338
6341
6338
6336
6334
6338
6335
6329
6330
6326
6325
6325
6327
6327
6330
6328
6327
6326
6324
6326
6323
6319
6314
6314
6312
6311
6311
6317
6314
6311
6308
6312
6308
6302
6300
6293
6295
6292
6288
6282
6278
6274
6272
6267
6263
6259
6259
6259
6255
6253
6252
6247
6256
6255
6261
6261
6262
6259
6259
6255
6252
6253
6246
6248
6244
6240
6241
6242
6242
6245
6244
6241
6242
6238
6236
6233
6226
6224
6222
6223
6220
6216
6218
6216
6216
6219
6215
6213
6217
6215
6212
6211
6209
6215
6218
6213
6213
6216
6219
6219
6223
6223
6222
6220
6220
6221
6221
6217
6217
6217
6217
6212
6213
6212
6215
6217
6206
6206
6204
6206
6207
6207
6205
6200
6205
6205
6210
6210
6217
6214
6218
6220
6222
6223
6225
6221
6217
6217
6213
6212
6214
6212
6207
6207
6204
6204
6203


5782
5781
5779
5780
5780
5779
5781
5782
5782
5782
5784
5783
5781
5781
5779
5777
5778
5778
5778
5774
5774
5774
5774
5774
5777
5777
5776
5773
5770
5768
5767
5767
5769
5769
5770
5770
5769
5769
5768
5767
5769
5772
5770
5768
5766
5766
5769
5766
5765
5765
5763
5761
5760
5761
5762
5763
5759
5758
5757
5753
5749
5750
5748
5749
5746
5743
5741
5736
5735
5735
5733
5729
5730
5733
5732
5729
5732
5733
5729
5728
5726
5723
5720
5718
5721
5722
5724
5725
5726
5721
5719
5719
5717
5716
5716
5715
5713
5712
5714
5714
5713
5712
5712
5712
5715
5714
5712
5711
5707
5712
5716
5715
5715
5715
5717
5716
5716
5716
5713
5715
5715
5718
5716
5719
5719
5718
5718
5716
5716
5715
5714
5714
5716
5712
5714
5714
5720
5716
5716
5717
5718
5719
5720
5719
5716
5714
5713
5712
5712
5710
5715
5715
5715
5714
5713
5712
5715
5716
5716
5717
5718
5716
5717
5717
5714
5715
5714
5713
5712
5709
5708
5709
5708
5708
5706
5705
5707
5706
5705
5705
5706
5703
5700
5700
5699
5695
5696
5696
5699
5699
5699
5699
5698
5697
5695
5701
5699
5697
5701
5701


5685
5685
5686
5688
5691
5692
5692
5695
5695
5695
5693
5692
5692
5689
5692
5694
5694
5697
5697
5698
5699
5699
5697
5697
5696
5699
5697
5695
5700
5701
5699
5700
5702
5706
5707
5707
5711
5710
5711
5712
5712
5711
5714
5714
5714
5714
5714
5712
5712
5712
5710
5709
5708
5708
5706
5706
5706
5704
5705
5697
5700
5701
5704
5704
5703
5704
5704
5706
5706
5706
5705
5702
5701
5701
5699
5700
5699
5697
5695
5695
5693
5693
5694
5694
5692
5692
5691
5690
5690
5691
5691
5692
5691
5690
5690
5690
5690
5689
5689
5689
5690
5690
5690
5689
5688
5687
5687
5687
5687
5687
5685
5686
5683
5683
5683
5683
5684
5683
5689
5689
5687
5685
5685
5683
5683
5683
5682
5682
5682
5681
5681
5681
5681
5680
5679
5679
5677
5679
5680
5680
5681
5681
5681
5681
5679
5677
5674
5674
5675
5675
5680
5680
5679
5674
5673
5673
5671
5673
5674
5673
5671
5680
5680
5682
5687
5689
5689
5692
5690
5692
5693
5691
5694
5693
5690
5690
5688
5688
5688
5691
5691
5690
5688
5686
5686
5689
5689
5689
5686
5685
5682
5681
5684
5686
5687
5689
5684
5686
5684
5683


5720
5720
5715
5716
5715
5714
5714
5712
5714
5711
5715
5716
5713
5713
5711
5709
5709
5709
5709
5709
5710
5715
5715
5714
5710
5713
5711
5711
5711
5710
5711
5710
5710
5710
5710
5708
5708
5708
5707
5705
5701
5701
5700
5699
5699
5699
5698
5698
5698
5698
5700
5701
5701
5701
5702
5703
5704
5705
5704
5703
5710
5710
5707
5707
5705
5702
5702
5702
5702
5701
5701
5703
5703
5703
5703
5703
5703
5701
5701
5699
5701
5700
5703
5702
5704
5704
5704
5704
5703
5703
5705
5705
5704
5706
5710
5719
5716
5716
5716
5715
5717
5720
5720
5719
5716
5715
5715
5715
5715
5715
5716
5716
5717
5717
5715
5717
5717
5715
5714
5712
5712
5715
5715
5713
5713
5713
5712
5712
5712
5715
5714
5713
5712
5711
5714
5714
5714
5715
5714
5715
5718
5718
5720
5720
5727
5727
5726
5726
5726
5723
5720
5718
5717
5721
5721
5722
5725
5725
5725
5725
5724
5723
5721
5720
5719
5717
5715
5715
5712
5711
5711
5711
5714
5711
5715
5715
5714
5714
5714
5714
5714
5714
5714
5713
5712
5712
5711
5711
5714
5713
5714
5712
5711
5711
5714
5714
5714
5713
5712
5712


5634
5632
5632
5632
5632
5631
5631
5631
5631
5630
5631
5633
5632
5630
5630
5628
5628
5627
5627
5627
5627
5626
5628
5626
5624
5620
5620
5619
5617
5617
5616
5617
5620
5619
5619
5619
5618
5617
5617
5619
5619
5618
5618
5620
5618
5617
5616
5616
5616
5616
5616
5616
5616
5616
5615
5615
5615
5608
5608
5606
5605
5605
5604
5604
5604
5604
5603
5603
5603
5603
5603
5603
5605
5603
5604
5602
5602
5602
5602
5602
5601
5601
5601
5599
5601
5603
5603
5603
5602
5601
5603
5600
5597
5597
5597
5595
5600
5600
5600
5600
5600
5601
5599
5598
5601
5600
5600
5600
5602
5602
5603
5603
5603
5603
5603
5606
5604
5603
5603
5603
5604
5604
5603
5602
5602
5602
5600
5600
5600
5599
5599
5599
5600
5596
5595
5596
5598
5598
5598
5597
5597
5595
5595
5595
5600
5600
5597
5597
5597
5597
5600
5599
5599
5601
5600
5600
5600
5598
5598
5598
5597
5597
5597
5595
5594
5593
5593
5590
5590
5590
5590
5589
5588
5588
5584
5584
5583
5581
5579
5579
5578
5579
5573
5575
5577
5576
5575
5574
5574
5572
5572
5570
5570
5570
5570
5570
5570
5568
5568
5568


5404
5404
5404
5404
5404
5403
5403
5403
5403
5402
5403
5403
5403
5406
5406
5408
5408
5408
5408
5408
5407
5407
5408
5408
5410
5410
5411
5413
5413
5413
5413
5413
5412
5411
5411
5412
5415
5414
5415
5416
5414
5414
5414
5414
5414
5414
5414
5414
5414
5412
5412
5410
5413
5413
5416
5416
5418
5417
5417
5416
5416
5416
5418
5418
5417
